In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 7.6 MB/s 


In [2]:
import socket
import sys
import requests
import requests_oauthlib
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [8]:
# Replace the values below with yours
ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
BEARER_TOKEN = ''


In [20]:
headers = {"Authorization": "Bearer {}".format(BEARER_TOKEN), "content-type": "application/json"}

In [122]:
def get_tweets():
    url = 'https://api.twitter.com/2/tweets/1228393702244134912'
    query_data = [('tweet.fields', 'created_at,attachments'), ('expansions', 'author_id')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, headers=headers, stream=True)
    print(query_url, response)
    
    return response

In [123]:
get_tweets()

https://api.twitter.com/2/tweets/1228393702244134912?tweet.fields=created_at,attachments&expansions=author_id <Response [200]>


<Response [200]>

In [93]:
def get_tweets_by_user(user):
    url = 'https://api.twitter.com/2/tweets/search/recent'
    query_data = [('query', 'from:'+user), ('max_results', '10')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, headers=headers, stream=True)
    #print(query_url, response)
    
    return response

In [110]:
def get_tweet_comments(id='1526022409383460866'):
    url = 'https://api.twitter.com/2/tweets/search/recent'
    #url = 'https://api.twitter.com/2/tweets/search/all'
    query_data = [('query', 'conversation_id:'+id), ('max_results', '100'), ('tweet.fields', 'conversation_id,in_reply_to_user_id,created_at')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, headers=headers, stream=True)
    #print(query_url, response)
    
    return response

In [95]:
analyzer = SentimentIntensityAnalyzer()

def process_tweets_comments(http_resp):
    data = http_resp.json()
    negative = 0
    neutral = 0
    positive = 0
    if 'data' in data:
      for comment in data['data']:
          tweet_text = str(comment['text'].encode("utf-8"))

          # analysis sentiment score
          sentiment_score = analyzer.polarity_scores(tweet_text)["compound"]
          if sentiment_score >= 0.05:
              sentiment = "POSITIVE"
              positive += 1
          elif sentiment_score <= -0.05:
              sentiment = "NEGATIVE"
              negative += 1
          else:
              sentiment = "NEUTRAL"
              neutral += 1

          # separate sentiment label with tweet content
          mess =  sentiment + '||||' + tweet_text + '\n'
          #print(mess)
          #tcp_connection.send(bytes(mess, 'utf-8'))
    return (positive,neutral,negative)

In [134]:
def analyze_sentimental_recent_post_of_user(username):
    print('Analyzing for user %s ...' %(username))
    resp_tw = get_tweets_by_user(username)
    data = resp_tw.json()
    #print(data)
    if 'data' in data:
        for tweet in data['data']:
            #print(tweet['id'])
            resp= get_tweet_comments(tweet['id'])
            #print(resp)
            positive,neutral,negative = process_tweets_comments(resp)
            print('TWEET: %s \nPOS: %d, NEU: %d, NEG: %d'%(tweet['text'],positive,neutral,negative))

In [109]:
resp= get_tweet_comments('1526022409383460866')
data = resp.json()
print(data)
#comments = data['data']
#positive,neutral,negative = process_tweets_comments(resp)

{'client_id': '24280907', 'detail': 'When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.', 'registration_url': 'https://developer.twitter.com/en/docs/projects/overview', 'title': 'Client Forbidden', 'required_enrollment': 'Standard Basic', 'reason': 'client-not-enrolled', 'type': 'https://api.twitter.com/2/problems/client-forbidden'}


In [135]:
analyze_sentimental_recent_post_of_user('POTUS')

Analyzing for user POTUS ...
TWEET: Tune in as I award the Public Safety Officer Medals of Valor for “extraordinary valor above and beyond the call of duty.” https://t.co/tCcfVXqqYJ 
POS: 17, NEU: 54, NEG: 29
TWEET: There is a quote engraved on the National Law Enforcement Memorial, “It is not how these officers died that made them heroes – it is how they lived.”
 
To the families, I know it’s hard. May the souls of those you love and with whom you served rest in peace and rise in glory. https://t.co/Lr4w737IwW 
POS: 12, NEU: 47, NEG: 41
TWEET: Jill and I will travel to Buffalo on Tuesday to grieve with the community that lost ten lives in a senseless and horrific mass shooting. 
POS: 19, NEU: 37, NEG: 44
TWEET: Jill and I pray for the victims, their families and devastated community from yesterday’s mass shooting in Buffalo, New York. Our hearts are heavy once again but our resolve must not waver; we must work together to address the hate that remains a stain on the soul of this natio